In [4]:
import mlflow
import os
import dagshub 

In [5]:
dagshub.init(repo_owner='sufyaansayed78', repo_name='Datascience', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Admin\Documents\Udemy MLops course\Datascience\venv\lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=3e1b4c60-742e-4e3d-8334-dc8e91177fc8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=acd44291e09472c37d87aac5ebc563681a3b8233d7f478e3453c12a9aa811995




Accessing as sufyaansayed78

Initialized MLflow to track repo "sufyaansayed78/Datascience"

Repository sufyaansayed78/Datascience initialized!

In [6]:
os.chdir("../")

In [9]:
from dataclasses import dataclass
from pathlib import Path

In [16]:
@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path 
    all_params : dict
    metric_file_name : Path 
    target_column : str 
    



In [17]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path= config.model_path,
            all_params = params ,
            metric_file_name= config.metric_file_name,
            target_column = schema.name)
        return model_evaluation_config
            







        



In [19]:
import os
import pandas as pd 
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np
import joblib 


In [24]:
class ModelEvaluation:
    def __init__(self,config : ModelEvaluationConfig):
        self.config = config 

    def eval_metrices(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column],axis =1)
        test_y = test_data[[self.config.target_column]]

        with mlflow.start_run():
          pred = model.predict(test_x)  
          rmse,mae,r2 = self.eval_metrices(test_y,pred=pred)
          scores = {"rmse": rmse,"mae":mae,"r2":r2}
          save_json(path=Path(self.config.metric_file_name),data=scores)
          mlflow.log_params(self.config.all_params)
          mlflow.log_metric("rmse",rmse)
          mlflow.log_metric("r2",r2)
          mlflow.log_metric("mae",mae)

        



        

In [25]:
try : 
    config  = ConfigurationManager()
    model_evaluation_config  = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e 

[2025-11-08 14:00:24,171: INFO: common:yaml file :config\config.yamlloaded successfully]
[2025-11-08 14:00:24,174: INFO: common:yaml file :params.yamlloaded successfully]
[2025-11-08 14:00:24,177: INFO: common:yaml file :schema.yamlloaded successfully]
[2025-11-08 14:00:24,178: INFO: common:Created direcotry at artifacts]
[2025-11-08 14:00:24,180: INFO: common:Created direcotry at artifacts/model_evaluation]
[2025-11-08 14:00:25,386: INFO: common:json file saved at : artifacts\model_evaluation\metrics.json]
🏃 View run thundering-fox-247 at: https://dagshub.com/sufyaansayed78/Datascience.mlflow/#/experiments/0/runs/1f22e7551a8c43e58eea70fcffd368a3
🧪 View experiment at: https://dagshub.com/sufyaansayed78/Datascience.mlflow/#/experiments/0
